In [1]:
from helper_functions import *
import nengo
import nengo_dl

In [2]:
inp = tf.keras.Input(shape=(4))
dense1 = layers.Dense(10, activation=tf.nn.relu, use_bias = True)(inp)
output = layers.Dense(1, use_bias = True)(dense1)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

In [3]:
onnx_model_path = '../Benchmarks/InvPend_Controller.onnx'
onnx_model = onnx.load(onnx_model_path)

weights_list=[]
i=0
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==1 and i >=0):
        weights_list.append(tensor_array)
    i = i+1
    
bias_list=[]
i=0
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==0 and i >0):
        bias_list.append(tensor_array)
    i = i+1
    
for i in range(1, len(weights_list) + 1):
    transposed_weights = np.transpose(weights_list[i-1])
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

In [4]:
weights, biases = extract_model_params_tf(nn_ctlr)

In [5]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings_5L_ACC_sim_random.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
    model=gp.Model("Encodings")

    model.Params.MIPFocus = focus
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1
    try:
        f = open(file_path,"r")
        try:
            for l in f:
                exec(l)
        finally:
            f.close()
    except IOError:
        pass
    model.optimize()
    
    return model

In [6]:
layer_no = 2
neuron_no = 1
time_steps = 5
input_bounds = [[-0.5,0.5],[0,0],[-0.2,0.2],[0,0]]

In [7]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]
    # Declarations
    for time in range(1,2):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")


    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")
           
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    
    equations.append(f"model.addConstr({out}>={cond[1]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MAXIMIZE)')


    return equations, declare

In [8]:
def can_go_below(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    # Declarations
    for time in range(1,2):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")

    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
            
    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    
    equations.append(f"model.addConstr({out}<={cond[0]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MINIMIZE)')


    return equations, declare

In [11]:
for tym in range(1,16):

    output_range = [-15.508829,  15.344650]
    print(output_range)
    print('\nLB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)

[-15.508829, 15.34465]

LB Checking

Time steps 1
4 Property holds
Runtime 0.002000093460083008
[-15.508829, 15.34465]

LB Checking

Time steps 2
4 Property holds
Runtime 0.0
[-15.508829, 15.34465]

LB Checking

Time steps 3
4 Property holds
Runtime 0.009999990463256836
[-15.508829, 15.34465]

LB Checking

Time steps 4
4 Property holds
Runtime 0.015999794006347656
[-15.508829, 15.34465]

LB Checking

Time steps 5
4 Property holds
Runtime 0.04499983787536621
[-15.508829, 15.34465]

LB Checking

Time steps 6
4 Property holds
Runtime 0.039999961853027344
[-15.508829, 15.34465]

LB Checking

Time steps 7
4 Property holds
Runtime 0.06200003623962402
[-15.508829, 15.34465]

LB Checking

Time steps 8
4 Property holds
Runtime 0.09500002861022949
[-15.508829, 15.34465]

LB Checking

Time steps 9
4 Property holds
Runtime 0.12199997901916504
[-15.508829, 15.34465]

LB Checking

Time steps 10
4 Property holds
Runtime 0.1700000762939453
[-15.508829, 15.34465]

LB Checking

Time steps 11
4 Property 

In [12]:
for tym in range(1,16):

    output_range = [-15.508829,  15.344650]
    print(output_range)
    print('\nUB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)

[-15.508829, 15.34465]

UB Checking

Time steps 1
4 Property holds
Runtime 0.0009999275207519531
[-15.508829, 15.34465]

UB Checking

Time steps 2
Property does not hold
Runtime 0.0019998550415039062
[-15.508829, 15.34465]

UB Checking

Time steps 3
4 Property holds
Runtime 0.0
[-15.508829, 15.34465]

UB Checking

Time steps 4
4 Property holds
Runtime 0.014999866485595703
[-15.508829, 15.34465]

UB Checking

Time steps 5
4 Property holds
Runtime 0.0
[-15.508829, 15.34465]

UB Checking

Time steps 6
4 Property holds
Runtime 0.04400014877319336
[-15.508829, 15.34465]

UB Checking

Time steps 7
4 Property holds
Runtime 0.014999866485595703
[-15.508829, 15.34465]

UB Checking

Time steps 8
4 Property holds
Runtime 0.015999794006347656
[-15.508829, 15.34465]

UB Checking

Time steps 9
4 Property holds
Runtime 0.05299997329711914
[-15.508829, 15.34465]

UB Checking

Time steps 10
4 Property holds
Runtime 0.04699993133544922
[-15.508829, 15.34465]

UB Checking

Time steps 11
4 Property holds


In [14]:
for tym in range(1,5):

    output_range = [-15.508829,  15.344650]
    print(output_range)
    print('\nUB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)

[-15.508829, 15.34465]

UB Checking

Time steps 1
4 Property holds
Runtime 0.0009999275207519531
[-15.508829, 15.34465]

UB Checking

Time steps 2
Property does not hold
Runtime 0.002000093460083008
[-15.508829, 15.34465]

UB Checking

Time steps 3
4 Property holds
Runtime 0.003000020980834961
[-15.508829, 15.34465]

UB Checking

Time steps 4
4 Property holds
Runtime 0.015000104904174805


In [30]:
for tym in range(2,4):

    output_range = [-15.508829,  15.344650]
    print(output_range)
    print('\nLB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)

[-15.508829, 15.34465]

LB Checking

Time steps 2
4 Property holds
Runtime 0.0
[-15.508829, 15.34465]

LB Checking

Time steps 3
4 Property holds
Runtime 0.012000083923339844


In [31]:
for tym in range(2,4):

    output_range = [-15.508829,  15.344650]
    print(output_range)
    print('\nUB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)

[-15.508829, 15.34465]

UB Checking

Time steps 2
Property does not hold
Runtime 0.0
[-15.508829, 15.34465]

UB Checking

Time steps 3
4 Property holds
Runtime 0.003000020980834961


In [36]:
output_range = [0,15.344650 + 0.2]
tym = 2
flag = 0
lb = 15.344650
ub = 15.344650  + 0.9
checked = []
result = []
while (flag <= 1):
#     mid = (lb+ub)/2
    print("LB:", lb)
    print("UB:", ub)
    mid = random.uniform(lb, ub)
    if(len(checked) > 1 and abs(ub - lb) < 0.001) :
        flag = flag + 1
        
        
    output_range[1] = mid
#     print(output_range)
    print("Checking with UB:", output_range[1])
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,1,6,0)
    checked.append(output_range[1])
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
        lb = mid 
    elif(model3.status in [9]):
        print('Time out')
        break
    else:
        print(model3.status, 'Property holds')
        ub = mid
    result.append(model3.status)
    print('Runtime',model3.Runtime)    
    print(checked)
    print(result)
    print("\n")

LB: 15.34465
UB: 16.24465
Checking with UB: 15.91319119629882
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 173 rows, 116 columns and 494 nonzeros
Model fingerprint: 0x8cd86ddf
Variable types: 76 continuous, 40 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-12, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [3e-01, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 141 rows and 95 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasi

Coefficient statistics:
  Matrix range     [3e-12, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [3e-01, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 141 rows and 95 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 2
4 Property holds
Runtime 0.01699995994567871
[15.91319119629882, 15.847125037648995, 15.574217437909594, 15.731886863760925, 15.71744853877727, 15.655541804966663, 15.638036415853012]
[4, 4, 2, 4, 4, 4, 4]


LB: 15.574217437909594
UB: 15.638036415853012
Checking with UB: 15.610607885069216
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(

Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 173 rows, 116 columns and 494 nonzeros
Model fingerprint: 0x0a05c1f1
Variable types: 76 continuous, 40 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-12, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [3e-01, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 173 rows and 116 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 31.2157 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.121570

  Matrix range     [3e-12, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [3e-01, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 173 rows and 116 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 31.2157 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.121570086479e+01, best bound 3.121570086479e+01, gap 0.0000%

Time steps 2
Property does not hold
Runtime 0.03600001335144043
[15.91319119629882, 15.847125037648995, 15.574217437909594, 15.731886863760925, 15.71744853877727, 15.655541804966663, 15.638036415853012, 15.610607885069216, 15.592549486646622, 15.599509919754276, 15.60513334041349, 15.606550330605097, 15.607311622408893, 15.60987290222251, 15.609712565599493, 15.609649634384075,